# Notebook to train PeakLens optimized model 

In [1]:
import sys
import os
import tensorflow as tf
import numpy as np
import glob
import multiprocessing

In [2]:
print(tf.__version__)

1.11.0


## Training parameters 

In [3]:
MODEL_NAME = "PeakLens_optimized"
DATASET_PATH = "/data/skyline-extraction-patches-dataset"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [4]:
BATCH_SIZE = 256
EPOCHS = 100
LEARNING_RATE = 0.001
EARLY_STOPPING = 10

## Model definition 

In [5]:
def deepnn(x):

    conv1 = tf.layers.conv2d(
        inputs=x,
        filters=32,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    conv2 = tf.layers.separable_conv2d(
        inputs=conv1,
        filters=32,
        kernel_size=[3, 3],
        strides=(2, 2),
        activation=tf.nn.relu)

    conv3 = tf.layers.separable_conv2d(
        inputs=conv2,
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    conv4 = tf.layers.separable_conv2d(
        inputs=conv3,
        filters=64,
        kernel_size=[3, 3],
        strides=(2, 2),
        activation=tf.nn.relu)

    conv5 = tf.layers.separable_conv2d(
        inputs=conv4,
        filters=128,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    conv6 = tf.layers.conv2d(
        inputs=conv5, 
        filters=2, 
        kernel_size=[3,3])

    output = tf.reshape(conv6, [-1, 1 * 1 * 2])

    softmax = tf.nn.softmax(output, name="softmax_tensor")

    return output, softmax

## Methods to load the dataset

In [6]:
def load_image_and_label(path, label):
    image = tf.read_file(path)
    image_decoded = tf.image.decode_jpeg(image)
    image_decoded = tf.image.random_flip_left_right(image_decoded)    
    return image_decoded, label

def get_dataset_split(split):
    positive_paths = glob.glob("{}/{}/patches/positive/*.jpg".format(DATASET_PATH, split))
    negative_paths = glob.glob("{}/{}/patches/negative/*.jpg".format(DATASET_PATH, split))
  
    positive_labels = [1] * len(positive_paths)
    negative_labels = [0] * len(negative_paths)
  
    paths = positive_paths + negative_paths
    labels = positive_labels + negative_labels

    tf_paths = tf.constant(paths)
    tf_labels = tf.constant(labels)

    dataset = tf.data.Dataset.from_tensor_slices((tf_paths, tf_labels))
    dataset = dataset.map(load_image_and_label, num_parallel_calls=multiprocessing.cpu_count())\
        .shuffle(len(paths)).batch(BATCH_SIZE).repeat(EPOCHS).prefetch(2)
    return dataset, len(paths)//BATCH_SIZE

## Load dataset splits

In [7]:
training_dataset, training_steps = get_dataset_split("training")
print("Training split loaded ({:,} images).".format(training_steps*BATCH_SIZE))

validation_dataset, validation_steps = get_dataset_split("validation")
print("Validation split loaded ({:,} images).".format(validation_steps*BATCH_SIZE))

test_dataset, test_steps = get_dataset_split("testing")
print("Test split loaded ({:,} images).".format(test_steps*BATCH_SIZE))

Training split loaded (1,699,328 images).
Validation split loaded (424,448 images).
Test split loaded (530,944 images).


## Build model graph

In [8]:
tf.reset_default_graph()

# Build the graph for the deep net
x = tf.placeholder(tf.float32, [None, 29, 29, 3])
y_ = tf.placeholder(tf.int64, [None])
y_conv, softmax = deepnn(x)

print("Parameters: {:,}.".format(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()])))

Parameters: 20,578.


## Define loss, adam_optimizer, accuracy

In [9]:
# Set cross entropy as cost function
with tf.name_scope('loss'):
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)

# Set Adam optimizer as trainer
with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

# Evaluation
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), y_)
    correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

# Training & Validation

It is possible to visualize the training with tensorboard by executing the following command with the corresponding logdir path

tensorboard --logdir=graphs

In [10]:
with tf.Session(config=config) as sess:

    sess.run(tf.global_variables_initializer())

    #Saver to save the best epoch (the one with lowest error)
    saver = tf.train.Saver()

    #Visualize on Tensorboard
    train_writer = tf.summary.FileWriter('./graphs/{}/train'.format(MODEL_NAME), sess.graph)
    validation_writer = tf.summary.FileWriter('./graphs/{}/validation'.format(MODEL_NAME), sess.graph)

    #Prepare iterator for the dataset
    training_iterator = training_dataset.make_one_shot_iterator()
    next_training_batch = training_iterator.get_next()

    validation_iterator = validation_dataset.make_one_shot_iterator()
    next_validation_batch = validation_iterator.get_next()

    print("Started training.")
    min_validation_loss = sys.maxint
    epochs_without_improvement = 0 # to determine whether early stopping is reached
    
    for epoch_i in range(EPOCHS):
        training_losses = []
        training_accuracies = []
        for i in range(training_steps):
            batch = sess.run(next_training_batch)

            accuracy_val, loss_val  = sess.run([accuracy, cross_entropy], feed_dict={x: batch[0], y_: batch[1]})
            training_accuracies.append(accuracy_val)
            training_losses.append(loss_val)
            
            train_step.run(feed_dict={x: batch[0], y_: batch[1]})
            
        epoch_accuracy = np.mean(training_accuracies)
        epoch_loss = np.mean(training_losses)
        
        print('Epoch %d, training accuracy: %g' % (epoch_i, epoch_accuracy))
        summary = tf.Summary()
        summary.value.add(tag="accuracy", simple_value=epoch_accuracy)
        summary.value.add(tag="loss", simple_value=epoch_loss)
        train_writer.add_summary(summary, global_step=epoch_i)

        validation_accuracies = []
        validation_losses = []
        for j in range(validation_steps):
            batch = sess.run(next_validation_batch)

            accuracy_val, loss_val = sess.run([accuracy, cross_entropy], feed_dict={x: batch[0], y_: batch[1]})
            validation_accuracies.append(accuracy_val)
            validation_losses.append(loss_val)
            
        epoch_accuracy = np.mean(validation_accuracies)
        epoch_loss = np.mean(validation_losses)        

        print('Epoch %d, validation accuracy: %g' % (epoch_i, epoch_accuracy))
        summary = tf.Summary()
        summary.value.add(tag="accuracy", simple_value=epoch_accuracy)
        summary.value.add(tag="loss", simple_value=epoch_loss)
        validation_writer.add_summary(summary, global_step=epoch_i)

        if(epoch_loss < min_validation_loss):
            min_validation_loss = epoch_loss
            epochs_without_improvement = 0
            print('Epoch %d, validation loss %g - session saved' % (epoch_i, min_validation_loss))
            saver.save(sess, "./checkpoint/{}.ckpt".format(MODEL_NAME))
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == EARLY_STOPPING:
                print('Training finished in epoch %d due to early stopping.' % epoch_i)
                break
    print('Finished training.')

Started training.
Epoch 0, training accuracy: 0.940754
Epoch 0, validation accuracy: 0.954395
Epoch 0, validation error 0.122062 - session saved
Epoch 1, training accuracy: 0.956603
Epoch 1, validation accuracy: 0.958634
Epoch 1, validation error 0.113437 - session saved
Epoch 2, training accuracy: 0.960124
Epoch 2, validation accuracy: 0.961347
Epoch 2, validation error 0.107196 - session saved
Epoch 3, training accuracy: 0.961666
Epoch 3, validation accuracy: 0.961418
Epoch 3, validation error 0.104961 - session saved
Epoch 4, training accuracy: 0.962832
Epoch 4, validation accuracy: 0.96282
Epoch 4, validation error 0.102163 - session saved
Epoch 5, training accuracy: 0.963649
Epoch 5, validation accuracy: 0.963092
Epoch 5, validation error 0.101458 - session saved
Epoch 6, training accuracy: 0.96421
Epoch 6, validation accuracy: 0.96228
Epoch 7, training accuracy: 0.964582
Epoch 7, validation accuracy: 0.963947
Epoch 7, validation error 0.0991454 - session saved
Epoch 8, training a

# Test

In [12]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    # Restore the best session
    saver.restore(sess, "./checkpoint/{}.ckpt".format(MODEL_NAME))
    
    # Compute test accuracy
    test_accuracies = []
    test_iterator = test_dataset.make_one_shot_iterator()
    next_test_batch = test_iterator.get_next()
    print("Started test.")
    for i in range(test_steps):
        batch = sess.run(next_test_batch)
        accuracy_val = sess.run([accuracy], feed_dict={x: batch[0], y_: batch[1]})
        test_accuracies.append(accuracy_val)

    print('Test accuracy: %g' % np.mean(test_accuracies))

INFO:tensorflow:Restoring parameters from ./checkpoint/PeakLens_optimized.ckpt
Started test.
Test accuracy: 0.963755


## Conversion to pb

In [14]:
tf.reset_default_graph()

# Define input size dimensions for complete images
x = tf.placeholder(tf.float32, [1, 240, 320, 3])
y_conv, softmax = deepnn(x)

saver = tf.train.Saver()

if not os.path.exists('./protobufs/'):
    os.makedir('./protobufs/')

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "./checkpoint/{}.ckpt".format(MODEL_NAME))
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, ["softmax_tensor"])
    
    with tf.gfile.GFile("./protobufs/{}.pb".format(MODEL_NAME), "wb") as f:
        f.write(output_graph_def.SerializeToString())
        print('Converted model to pb.')

INFO:tensorflow:Restoring parameters from ./checkpoint/PeakLens_optimized.ckpt
INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.
Converted model to pb.
